In [23]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn
import inflect

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import LSTM, RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import RMSprop

In [2]:
p = inflect.engine()

In [3]:
pairs = {}
for synset in wn.all_synsets('n'):
    word = synset.name().split('.', 1)[0]
    if not word in pairs:
        pairs[word] = p.plural(word)
len(pairs)

67176

In [4]:
with open('data/plurals.txt', 'w') as fout:
    for k in sorted(pairs):
        if '_' in k or '-' in k:
            continue
        if k.isdigit():
            continue
        fout.write('%s\t%s\n' % (k, pairs[k]))

In [5]:
p.plural('no')

'noes'

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [8]:
# Parameters for the model and dataset.
INVERT = True

In [9]:
questions = []
expected = []
seen = set()
#with open('data/en_de.txt') as fin:
with open('data/plurals.txt') as fin:
    for line in fin:
        en, de = line.strip().split('\t')
        questions.append(en)
        expected.append(de)

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Total addition questions: 39929


In [10]:
chars = set(ch for k, v in zip(questions, expected) for ch in k + v)
ctable = CharacterTable(chars)
len(chars)

40

In [11]:
print('Vectorization...')
x = np.zeros((len(questions), max_question_len, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), max_expected_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, max_question_len)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, max_expected_len)
print('done')

Vectorization...
done


In [12]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Training Data:
(35937, 31, 40)
(35937, 32, 40)
Validation Data:
(3992, 31, 40)
(3992, 32, 40)


In [13]:
# The below is taken from: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(max_question_len, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.Dropout(DROP_OUT))
model.add(layers.RepeatVector(max_expected_len))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
#    model.add(layers.Dropout(DROP_OUT))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   (None, 128)               86528     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 32, 128)           0         
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (None, 32, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 40)            5160      
_________________________________________________________________
activation (Activation)      (None, 32, 40)            0         
Total params: 223,272
Trainable params: 223,272
Non-trainable params: 0
_________________________________________________________________


In [24]:
def create_seq2seq(num_nodes, num_layers):
    num_chars = len(chars)
    question = Input(shape=(max_question_len, num_chars), name='question')
    # repeat = RepeatVector(max_expected_len)(question)
    prev = question
    for _ in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name='lstm_layer_%d' % (i + 1))(prev)
        prev = lstm
    dense = TimeDistributed(Dense(num_chars, name='dense', activation='softmax'))(prev)
    model = Model(inputs=[question], outputs=[dense])
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seq2seq = create_seq2seq(128, 1)

In [25]:

BATCH_SIZE = 2048

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)


Train on 35937 samples, validate on 3992 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
35937/35937 [==============================] - 6s 178us/sample - loss: 2.1031 - acc: 0.6659 - val_loss: 1.3579 - val_acc: 0.7055
Epoch 2/10
35937/35937 [==============================] - 1s 40us/sample - loss: 1.2312 - acc: 0.7062 - val_loss: 1.0942 - val_acc: 0.7070
Epoch 3/10
35937/35937 [==============================] - 1s 39us/sample - loss: 1.0432 - acc: 0.7161 - val_loss: 0.9938 - val_acc: 0.7200
Epoch 4/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.9380 - acc: 0.7248 - val_loss: 0.8858 - val_acc: 0.7275
Epoch 5/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.8900 - acc: 0.7307 - val_loss: 0.8591 - val_acc: 0.7406
Epoch 6/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.8502 - acc: 0.7396 - val_loss: 0.8392 - val_ac

35937/35937 [==============================] - 1s 40us/sample - loss: 0.7817 - acc: 0.7597 - val_loss: 0.7752 - val_acc: 0.7611
Epoch 2/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7726 - acc: 0.7631 - val_loss: 0.7738 - val_acc: 0.7621
Epoch 3/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7711 - acc: 0.7638 - val_loss: 0.7736 - val_acc: 0.7611
Epoch 4/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7702 - acc: 0.7643 - val_loss: 0.7714 - val_acc: 0.7635
Epoch 5/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7694 - acc: 0.7647 - val_loss: 0.7710 - val_acc: 0.7626
Epoch 6/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7688 - acc: 0.7649 - val_loss: 0.7701 - val_acc: 0.7630
Epoch 7/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.8405 - acc: 0.7434 - val_loss: 0.8054 - val_acc: 0.7429
Epoch 8/10
35937/35937 [==============

35937/35937 [==============================] - 1s 40us/sample - loss: 0.7982 - acc: 0.7558 - val_loss: 0.8733 - val_acc: 0.7139
Epoch 6/10
35937/35937 [==============================] - 1s 39us/sample - loss: 0.8250 - acc: 0.7399 - val_loss: 0.7962 - val_acc: 0.7572
Epoch 7/10
35937/35937 [==============================] - 1s 39us/sample - loss: 0.7736 - acc: 0.7607 - val_loss: 0.7642 - val_acc: 0.7650
Epoch 8/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7570 - acc: 0.7680 - val_loss: 0.7554 - val_acc: 0.7695
Epoch 9/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7526 - acc: 0.7709 - val_loss: 0.7531 - val_acc: 0.7706
Epoch 10/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7506 - acc: 0.7714 - val_loss: 0.7515 - val_acc: 0.7700

--------------------------------------------------
Iteration 7
                      prognosis (prognoses                       ) - carriiiies                      
          

35937/35937 [==============================] - 1s 40us/sample - loss: 0.7939 - acc: 0.7488 - val_loss: 0.7778 - val_acc: 0.7581
Epoch 10/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7624 - acc: 0.7620 - val_loss: 0.7561 - val_acc: 0.7629

--------------------------------------------------
Iteration 10
                       frotteur (frotteurs                       ) - sareiiies                       
              mineralocorticoid (mineralocorticoids              ) - cereeeeeoiiiiiiies              
                            two (twoes                           ) - saass                           
                         operon (operons                         ) - sariies                         
                 automysophobia (automysophobias                 ) - cereeoooiiiiies                 
                    habituation (habituations                    ) - ceroooiiiies                    
                          skunk (skunks                

35937/35937 [==============================] - 1s 39us/sample - loss: 0.7304 - acc: 0.7771 - val_loss: 0.7295 - val_acc: 0.7774
Epoch 2/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7272 - acc: 0.7780 - val_loss: 0.7281 - val_acc: 0.7774
Epoch 3/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7264 - acc: 0.7784 - val_loss: 0.7276 - val_acc: 0.7775
Epoch 4/10
35937/35937 [==============================] - 1s 40us/sample - loss: 0.7518 - acc: 0.7683 - val_loss: 0.8604 - val_acc: 0.7496
Epoch 5/10
35937/35937 [==============================] - 1s 41us/sample - loss: 0.7941 - acc: 0.7524 - val_loss: 0.7755 - val_acc: 0.7584
Epoch 6/10
35937/35937 [==============================] - 1s 41us/sample - loss: 0.7539 - acc: 0.7683 - val_loss: 0.7396 - val_acc: 0.7727
Epoch 7/10
35937/35937 [==============================] - 1s 41us/sample - loss: 0.7347 - acc: 0.7753 - val_loss: 0.7305 - val_acc: 0.7758
Epoch 8/10
35937/35937 [==============

KeyboardInterrupt: 

In [27]:
shakespeare = strip_headers(load_etext(100))
tokens = [tuple(word) for word in tokenize(shakespeare, to_lower=True)]
# tokens = [tuple(word) for word in tokenize(plays, to_lower=True)]
token_counts = Counter(tokens)

In [28]:
pairs = [(token[i], token[i + 1], token_id) for token_id, token in enumerate(tokens) for i in range(len(token) - 1)]

In [29]:
pairs[10]

('o', 'r', 2)

In [30]:
help(tokenize)

Help on function tokenize in module gensim.utils:

tokenize(text, lowercase=False, deacc=False, encoding='utf8', errors='strict', to_lower=False, lower=False)
    Iteratively yield tokens as unicode strings, removing accent marks
    and optionally lowercasing the unidoce string by assigning True
    to one of the parameters, lowercase, to_lower, or lower.
    
    Input text may be either unicode or utf8-encoded byte string.
    
    The tokens on output are maximal contiguous sequences of alphabetic
    characters (no digits!).
    
    >>> list(tokenize('Nic nemůže letět rychlostí vyšší, než 300 tisíc kilometrů za sekundu!', deacc = True))
    [u'Nic', u'nemuze', u'letet', u'rychlosti', u'vyssi', u'nez', u'tisic', u'kilometru', u'za', u'sekundu']



In [31]:
tokens[5]

('s', 'h', 'a', 'k', 'e', 's', 'p', 'e', 'a', 'r', 'e')